# Build (MGP)
Using an Area of Interest (AOI) from the `aoi` SQLite Database, queries the Maxar Geospatial Platform API for data over a given time period, builds the `mgp` table from these results, and then populates the table with these values. Other notebooks should be used for adding additional records to the table.

### Import libraries

In [1]:
# Basic stack
from datetime import datetime

# Web Stack
import json
import requests

# Database stack
import sqlite3

# Data Science stack
import shapely
from shapely.geometry import box, Polygon
import pandas as pd
import geopandas as gpd
import folium

# Custom stack
import sys; sys.path.append("../../")
from MGP import security, search

### User defined variables

In [2]:
aoi_id = '6'
collections = ["wv03-vnir"]
start_date = '2014-08-13'
end_date = '2024-05-31'
where_clause = "eo:cloud_cover <= 90 and off_nadir_avg <= 90"
limit = 1_000
db = "C:/gis/gaia/data/databases/gaia.db"

### User defined functions

In [3]:
def update_mgp(c, entity_id, column_name, column_value):
    """ Updates EE table values of a given column with a given value using the 
            ENTITY ID as the selection criteria for the update.

        C - A cursor
        ENTITY ID - An entity id from EarthExplorer
        COLUMN NAME - A column name within the EE table to be updated
        COLUMN VALUE - A value to update the column
    """
    force_string_list = ['platform', 'instruments']
    force_date_list2 = ['datetime']
    force_geom_list = ['bbox']
    
    if column_name in force_string_list:
        sql_string = f"UPDATE mgp SET {column_name} = \"{column_value}\" WHERE id = \"{entity_id}\""
        c.execute(sql_string)
    
    elif column_name in force_date_list2:
        sql_string = f"UPDATE mgp SET {column_name} = "
        sql_string = sql_string + "? WHERE id = ?"
        c.execute(sql_string, (column_value, entity_id))
    
    elif column_name in force_geom_list:
        sql_string = f"UPDATE mgp SET {column_name} = GeomFromText(\"{column_value}\") WHERE id = \"{entity_id}\""
        c.execute(sql_string)
    
    else:
        sql_string = f"UPDATE mgp SET {column_name} = {column_value} WHERE id = \"{entity_id}\""
        c.execute(sql_string)

def database_activity(db, entity_id, column_name, column_value, activity='update'):
    """ Wrapper function for activities related to database management.
            Currently only supports UPDATING based on EarthExplorer 
            Entity ID values.

        Is dependent on the UPDATE EE function above.

        DB - A database target
        ENTITY ID - An EarthExplorer Entity ID uniquely identifying
            the record of interest.
        COLUMN NAME - The column to be updated
        COLUMN VALUE - The value of the column to be updated
        ACTIVITY - The activity to be carried out. Currently only
            supports UPDATE.
    """
    conn = sqlite3.connect(db)
    conn.enable_load_extension(True)
    conn.execute("SELECT load_extension('mod_spatialite')")
    c = conn.cursor()

    if activity == 'update':
        try:
            update_mgp(c, entity_id, column_name, column_value)
        except Exception as e:
            print("Exception: {} was raised for Entity ID {}".format(e, row['entity_id']))
    else:
        print("Your activity method is not currently supported.")
    
    conn.commit()
    conn.close()

def insert_mgp(c, iid):
    """ Inserts Maxar Geospatial Platform IDs into the MGP Data Table.
            The IDs are the Primary Key for the datatable since it is
            unique

        C - A cursor
        ID - An ID value from Maxar Geospatial Platform
    """
    sql_string = f"INSERT INTO mgp(id) VALUES (\"{iid}\")"
    c.execute(sql_string)

### Get MGP Token
Hint: ###

In [6]:
token_file = security.fetch_auth()

Enter your username:   john.wall@noaa.gov
Enter your password:   ········


Writing file now!


### Retreve the Area of Interest record from the `aoi` table corresponding to the supplied DAR number

In [8]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

sql_string = f"SELECT aid, name, sqkm, AsText(geom) FROM aoi WHERE aid={aoi_id}"
df = pd.read_sql_query(sql_string, conn)
df = df.rename(columns={'AsText(geom)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf_aoi = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

gdf_aoi

,aid,name,sqkm,geometry
0,6,Cape Cod Bay,1347.04,"POLYGON ((-70.11938 42.10724, -70.52387 42.109..."


### Search MGP

In [9]:
gdf_json = gdf_aoi['geometry'].to_json()

response, id_list = search.search(
    collections = collections,
    start = start_date,
    end = end_date,
    where = where_clause,
    geometry = gdf_json,
    limit = limit
)

print("Here are some example ImageIDs your query returned: {}".format(id_list[0:5]))

{
  "datetime": "2014-08-13/2024-05-31",
  "where": "eo:cloud_cover <= 90 and off_nadir_avg <= 90",
  "intersects": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -70.119385,
          42.107239
        ],
        [
          -70.523865,
          42.109436
        ],
        [
          -70.526001,
          41.747925
        ],
        [
          -70.121399,
          41.745544
        ],
        [
          -70.119385,
          42.107239
        ]
      ]
    ]
  },
  "limit": 1000,
  "page": 1,
  "collections": [
    "wv03-vnir"
  ]
}
Total of 168 objects returned
Here are some example ImageIDs your query returned: ['1040010093625500', '10400100959B5400', '1040010096672600', '1040010091057E00', '1040010093A5AA00']


In [10]:
response.json()

{'type': 'FeatureCollection',
 'features': [{'id': '1040010093625500',
   'bbox': [-70.613369, 41.780362, -70.333672, 42.162377],
   'type': 'Feature',
   'links': [{'rel': 'collection',
     'href': 'https://api.maxar.com/discovery/v1/collections/wv03-vnir'},
    {'rel': 'self',
     'href': 'https://api.maxar.com/discovery/v1/collections/wv03-vnir/items/1040010093625500'},
    {'rel': 'root', 'href': 'https://api.maxar.com/discovery/v1/'},
    {'rel': 'order-map-ready',
     'href': 'https://api.maxar.com/ordering/v1/pipelines/imagery/map-ready/order',
     'type': 'application/json'},
    {'rel': 'order-system-ready',
     'href': 'https://api.maxar.com/ordering/v1/pipelines/imagery/system-ready/order',
     'type': 'application/json'},
    {'rel': 'order-view-ready',
     'href': 'https://api.maxar.com/ordering/v1/pipelines/imagery/view-ready/order',
     'type': 'application/json'},
    {'rel': 'order-view-ready-ortho',
     'href': 'https://api.maxar.com/ordering/v1/pipelines/ima

### Create a GeoDataFrame from query results

In [11]:
columns = ["id", "platform", "instruments", "gsd", "pan_resolution_avg",
           "multi_resolution_avg", "datetime", "off_nadir", "azimuth",
           "sun_azimuth", "sun_elevation", "bbox"]
gdf = gpd.GeoDataFrame(columns = columns)
gdf = gdf.set_geometry("bbox").set_crs("EPSG:4326")

r = response.json()
for i, feature in enumerate(r["features"]):
    gdf.loc[i, "id"] = feature["id"]
    gdf.loc[i, "aoi_id"] = aoi_id
    gdf.loc[i, "platform"] = feature["properties"]["platform"]
    gdf.loc[i, "instruments"] = ', '.join(feature["properties"]["instruments"])
    gdf.loc[i, "gsd"] = feature["properties"]["gsd"]
    gdf.loc[i, "pan_resolution_avg"] = feature["properties"]["pan_resolution_avg"]
    gdf.loc[i, "multi_resolution_avg"] = feature["properties"]["multi_resolution_avg"]
    gdf.loc[i, "datetime"] = feature["properties"]["datetime"]
    gdf.loc[i, "off_nadir"] = feature["properties"]["view:off_nadir"]
    gdf.loc[i, "azimuth"] = feature["properties"]["view:azimuth"]
    gdf.loc[i, "sun_azimuth"] = feature["properties"]["view:sun_azimuth"]
    gdf.loc[i, "sun_elevation"] = feature["properties"]["view:sun_elevation"]
    gdf.loc[i, "bbox"] = box(*feature["bbox"])

gdf.head()

,id,platform,instruments,gsd,pan_resolution_avg,multi_resolution_avg,datetime,off_nadir,azimuth,sun_azimuth,sun_elevation,bbox,aoi_id
0,1040010093625500,worldview-03,VNIR,0.48959,0.48959,1.957822,2024-05-10T15:12:09.830632Z,38.731296,298.21721,135.97089,59.840196,"POLYGON ((-70.33367 41.78036, -70.33367 42.162...",6
1,10400100959B5400,worldview-03,VNIR,0.465732,0.465732,1.861867,2024-05-10T15:11:58.030657Z,36.9672,290.060698,136.29946,59.933401,"POLYGON ((-70.09283 41.80274, -70.09283 42.154...",6
2,1040010096672600,worldview-03,VNIR,0.387751,0.387751,1.566842,2024-05-04T15:18:49.787012Z,28.198752,300.877281,140.548459,59.617618,"POLYGON ((-69.99209 41.46543, -69.99209 42.029...",6
3,1040010091057E00,worldview-03,VNIR,0.477092,0.477092,1.825612,2024-03-14T15:21:50.586824Z,37.508765,332.149348,149.478354,41.594089,"POLYGON ((-70.44030 41.77703, -70.44030 42.178...",6
4,1040010093A5AA00,worldview-03,VNIR,0.400867,0.400867,1.545379,2024-03-14T15:21:27.286870Z,29.931199,320.08349,149.822968,41.697987,"POLYGON ((-70.07881 41.79959, -70.07881 42.149...",6


In [12]:
gdf.shape

(168, 13)

### Drop table
This is for demonstration purposes

In [13]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS mgp''')
conn.commit()
conn.close()

### Create `mgp` table

In [14]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS mgp(
        id VARCHAR(16) PRIMARY KEY,
        aoi_id VARCHAR(4),
        platform VARCHAR(11),
        instruments VARCHAR(4),
        gsd NUMERIC(2, 3),
        pan_resolution_avg NUMERIC(2, 3),
        multi_resolution_avg NUMERIC(2, 3),
        datetime DATETIME2,
        off_nadir NUMERIC(2, 3),
        azimuth NUMERIC(3, 3),
        sun_azimuth NUMERIC(3, 3),
        sun_elevation NUMERIC(3, 3),
        bbox POLYGON,
        FOREIGN KEY (aoi_id)
            REFERENCES aoi(id)
    )
''')

conn.commit()
conn.close()

### Insert Maxar Geospatial Platform IDs into `mgp` table

In [15]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in gdf.iterrows():
    try:
        insert_mgp(c, row['id'])
    except Exception as e:
        print("Exception: {} was raised for ID {}".format(e, row['id']))
print("Done inserting Imagery IDs into table!")
conn.commit()
conn.close()

Done inserting Imagery IDs into table!


### Update `mgp` records from Maxar Geospatial Platform results

In [16]:
columns = gdf.columns[1:]
for i, row in gdf.iterrows():
    iid = row['id']
    print("Updating information for Imagery ID: {}".format(iid))
    for column in columns:
        database_activity(db, iid, column, row[column])

Updating information for Imagery ID: 1040010093625500
Updating information for Imagery ID: 10400100959B5400
Updating information for Imagery ID: 1040010096672600
Updating information for Imagery ID: 1040010091057E00
Updating information for Imagery ID: 1040010093A5AA00
Updating information for Imagery ID: 1040010093467000
Updating information for Imagery ID: 1040010093847B00
Updating information for Imagery ID: 1040010092BD6D00
Updating information for Imagery ID: 10400100928B3000
Updating information for Imagery ID: 10400100915D6100
Updating information for Imagery ID: 104001009037FF00
Updating information for Imagery ID: 104001008D3FC800
Updating information for Imagery ID: 104001009076F600
Updating information for Imagery ID: 104001008C471A00
Updating information for Imagery ID: 104001008D113000
Updating information for Imagery ID: 104001008D95DA00
Updating information for Imagery ID: 104001008C0F7C00
Updating information for Imagery ID: 104001008DC4AA00
Updating information for Ima

### Select newly inserted AOIs, make a GeoDataFrame for validation

In [17]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

columns_list = list(gdf.columns[:-1])
columns_str = ', '.join(columns_list)
sql_string = "SELECT {}, AsText(bbox) FROM mgp WHERE id IS NOT NULL AND aoi_id = {}".format(columns_str, aoi_id)

df = pd.read_sql_query(sql_string, conn)
df = df.rename(columns={'AsText(bbox)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

gdf.head()

,id,platform,instruments,gsd,pan_resolution_avg,multi_resolution_avg,datetime,off_nadir,azimuth,sun_azimuth,sun_elevation,bbox,geometry
0,1040010093625500,worldview-03,VNIR,0.489590,0.489590,1.957822,2024-05-10T15:12:09.830632Z,38.731296,298.217210,135.970890,59.840196,b'\x00\x01\x00\x00\x00\x00V\xd8\x0cpA\xa7Q\xc0...,"POLYGON ((-70.33367 41.78036, -70.33367 42.162..."
1,10400100959B5400,worldview-03,VNIR,0.465732,0.465732,1.861867,2024-05-10T15:11:58.030657Z,36.967200,290.060698,136.299460,59.933401,b'\x00\x01\x00\x00\x00\x00\xb5\xc0\x1e\x13)\x9...,"POLYGON ((-70.09283 41.80274, -70.09283 42.154..."
2,1040010096672600,worldview-03,VNIR,0.387751,0.387751,1.566842,2024-05-04T15:18:49.787012Z,28.198752,300.877281,140.548459,59.617618,b'\x00\x01\x00\x00\x00\x00\xe0\xf3\xc3\x08\xe1...,"POLYGON ((-69.99209 41.46543, -69.99209 42.029..."
3,1040010091057E00,worldview-03,VNIR,0.477092,0.477092,1.825612,2024-03-14T15:21:50.586824Z,37.508765,332.149348,149.478354,41.594089,b'\x00\x01\x00\x00\x00\x00\x89\xb2\xb7\x94\xf3...,"POLYGON ((-70.44030 41.77703, -70.44030 42.178..."
4,1040010093A5AA00,worldview-03,VNIR,0.400867,0.400867,1.545379,2024-03-14T15:21:27.286870Z,29.931199,320.083490,149.822968,41.697987,b'\x00\x01\x00\x00\x00\x00\xcdV^\xf2?\x92Q\xc0...,"POLYGON ((-70.07881 41.79959, -70.07881 42.149..."


In [18]:
# Note that the GDF shape matches that from the above
gdf.shape

(168, 13)

### Plot Areas of Interest on an Interactive Map

In [19]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

folium.GeoJson(
    gdf_json,
    style_function = lambda x: style_function('#FF0000')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

# End